In [1]:
import pandas as pd
from sklearn.externals import joblib as jl
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import tldextract

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import igraph as ig

pd.set_option('display.max_columns', 50)
%matplotlib inline

In [2]:
# data = pd.read_csv('data/seo-search-engine-optimization-ke-metrics-serps-extended-12-Dec-9cd971c1cb802b0083b1c01ccb86b37a.csv')
# data = data[(data['Keyword'] == data['Top Keyword'])]
# # data['URL'] = data.URL.apply(lambda x: x.rstrip('/'))
# data.head()

In [3]:
# def get_backlinks (url_list, backlink_list):
#     token = 'bd784d863a1d6a7a6d377d7482cec31f60eed50b'
#     backlinks_list = []
#     for url, lim in zip(url_list, backlink_list):
#         url = url.replace(':','%3A').replace('/', '%2F')
#         r = requests.get("""http://apiv2.ahrefs.com?token={}&target={}&output=json&from=backlinks&mode=exact&limit={}""".format(token, url, lim) )
#         try:
#             backlinks = pd.DataFrame(r.json()['refpages'])
#             backlinks_list.append(backlinks)
#         except ValueError:  # includes simplejson.decoder.JSONDecodeError
#             print ('Decoding JSON has failed for {}'.format(url))

#     return pd.concat(backlinks_list)

In [4]:
# %%time
# backlinks = get_backlinks(data['URL'].tolist(), data['Backlinks'].tolist())

In [5]:
# jl.dump(backlinks, 'data/seo_backlinks.jl')

In [6]:
backlinks = jl.load('data/seo_backlinks.jl')

In [7]:
backlinks.head()

,ahrefs_rank,ahrefs_top,alt,anchor,domain_rating,encoding,first_origin,first_seen,http_code,ip_from,language,last_origin,last_visited,link_type,links_external,links_internal,nofollow,original,page_size,prev_visited,redirect,sitewide,text_post,text_pre,title,url_from,url_from_first_seen,url_to
0,6,0,,What Is SEO / Search Engine Optimization?,36,utf8,recrawl,2017-10-31T00:15:34Z,200,81.223.238.246,en,recrawl,2017-10-31T00:15:34Z,href,50,11,False,False,5982,2017-08-14T23:42:54Z,0,False,,,Austrobot Österreich Suchmaschine,http://austrobot.at/Videos/search-engine-adver...,2017-01-24T07:09:07Z,https://searchengineland.com/guide/what-is-seo
1,6,0,,What Is SEO / Search Engine Optimization?,36,utf8,recrawl,2017-11-08T02:05:05Z,200,81.223.238.246,en,recrawl,2017-11-08T02:05:05Z,href,50,11,False,False,6005,2017-09-30T15:00:26Z,0,False,,,Austrobot Österreich Suchmaschine,http://austrobot.at/Videos/seo-definition,2017-01-18T02:11:22Z,https://searchengineland.com/guide/what-is-seo
2,6,0,,What Is SEO / Search Engine Optimization?,36,utf8,recrawl,2017-11-19T03:33:41Z,200,81.223.238.246,de,recrawl,2017-11-19T03:33:41Z,href,50,11,False,False,6074,2017-10-07T00:07:18Z,0,False,,,Austrobot Österreich Suchmaschine,http://austrobot.at/ar/Videos/seo-definition,2017-01-15T03:16:40Z,https://searchengineland.com/guide/what-is-seo
3,6,0,,What Is SEO / Search Engine Optimization?,36,utf8,recrawl,2017-11-17T16:30:46Z,200,81.223.238.246,fr,recrawl,2017-11-17T16:30:46Z,href,44,10,False,False,5826,2017-10-03T13:31:47Z,0,False,,,Austrobot Österreich Suchmaschine,http://austrobot.at/ar/Videos/seo-sea-definition,2017-02-16T03:13:03Z,https://searchengineland.com/guide/what-is-seo
4,6,0,,What Is SEO / Search Engine Optimization?,36,utf8,recrawl,2017-10-23T07:19:42Z,200,81.223.238.246,de,recrawl,2017-11-28T11:40:22Z,href,50,11,False,False,5983,2017-10-23T07:19:43Z,0,False,,,Austrobot Österreich Suchmaschine,http://austrobot.at/de/Videos/seo-definition,2017-01-18T08:57:57Z,https://searchengineland.com/guide/what-is-seo


In [8]:
links = backlinks[backlinks.nofollow == False][['url_from', 'url_to']].reset_index(drop=True)
links.head()

,url_from,url_to
0,http://austrobot.at/Videos/search-engine-adver...,https://searchengineland.com/guide/what-is-seo
1,http://austrobot.at/Videos/seo-definition,https://searchengineland.com/guide/what-is-seo
2,http://austrobot.at/ar/Videos/seo-definition,https://searchengineland.com/guide/what-is-seo
3,http://austrobot.at/ar/Videos/seo-sea-definition,https://searchengineland.com/guide/what-is-seo
4,http://austrobot.at/de/Videos/seo-definition,https://searchengineland.com/guide/what-is-seo


In [9]:
links['url_from'] = links.url_from.apply(lambda x: tldextract.extract(x).registered_domain)
links['url_to'] = links.url_to.apply(lambda x: tldextract.extract(x).registered_domain)
links = links[(links.url_from != '') & (links.url_to != '')]
links = links.drop_duplicates().reset_index(drop=True)

links.head()

,url_from,url_to
0,austrobot.at,searchengineland.com
1,advis.com.au,searchengineland.com
2,duxdigital.com.au,searchengineland.com
3,getwiththebrand.com.au,searchengineland.com
4,webjex.com.au,searchengineland.com


In [10]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21156 entries, 0 to 21155
Data columns (total 2 columns):
url_from    21156 non-null object
url_to      21156 non-null object
dtypes: object(2)
memory usage: 330.6+ KB


In [11]:
grouped_src_dst = links.groupby(["url_from","url_to"]).size().reset_index()

In [12]:
unique_links = pd.Index(grouped_src_dst['url_from']
                      .append(grouped_src_dst['url_to'])
                      .reset_index(drop=True).unique())

In [13]:
group_dict = {}
counter = 0
for lk in unique_links:
    if lk not in group_dict:
        counter += 1
        group_dict[lk] = counter
    else:
        pass

In [14]:
grouped_src_dst.rename(columns={0:'count'}, inplace=True)
temp_links_list = list(grouped_src_dst.apply(lambda row: {"source": row['url_from'],
                                                          "target": row['url_to'],
                                                          "value": row['count']}, axis=1))

In [15]:
temp_links_list[:5]

[{'source': '000.nl', 'target': 'moz.com', 'value': 1},
 {'source': '01webdirectory.com', 'target': 'google.com', 'value': 1},
 {'source': '024a.com.cn', 'target': 'moz.com', 'value': 1},
 {'source': '027safe.com', 'target': 'google.com', 'value': 1},
 {'source': '0531w.com', 'target': 'searchenginewatch.com', 'value': 1}]

In [16]:
len(unique_links)

18436

In [17]:
links_list = []
for link in temp_links_list:
    record = {"value":link['value'], "source":unique_links.get_loc(link['source']),
     "target": unique_links.get_loc(link['target'])}
    links_list.append(record)

In [18]:
nodes_list = []

for lk in unique_links:
    nodes_list.append({"name":lk, "group": group_dict.get(lk)})

In [19]:
links_list[:5]

[{'source': 0, 'target': 10470, 'value': 1},
 {'source': 1, 'target': 6456, 'value': 1},
 {'source': 2, 'target': 10470, 'value': 1},
 {'source': 3, 'target': 6456, 'value': 1},
 {'source': 4, 'target': 13593, 'value': 1}]

In [20]:
nodes_list[:5]

[{'group': 1, 'name': '000.nl'},
 {'group': 2, 'name': '01webdirectory.com'},
 {'group': 3, 'name': '024a.com.cn'},
 {'group': 4, 'name': '027safe.com'},
 {'group': 5, 'name': '0531w.com'}]

In [21]:
json_prep = {"links":links_list, "nodes":nodes_list}

json_prep.keys()

dict_keys(['nodes', 'links'])

In [22]:
json_dump = json.dumps(json_prep, indent=1, sort_keys=True)

In [23]:
pd.DataFrame(json_prep['nodes']).head()

,group,name
0,1,000.nl
1,2,01webdirectory.com
2,3,024a.com.cn
3,4,027safe.com
4,5,0531w.com


In [ ]:
pd.DataFrame(json_prep['links']).head()

,source,target,value
0,0,10470,1
1,1,6456,1
2,2,10470,1
3,3,6456,1
4,4,13593,1


In [ ]:
N=len(json_prep['nodes'])

L=len(json_prep['links'])
Edges=[(json_prep['links'][k]['source'], json_prep['links'][k]['target']) for k in range(L)]

G=ig.Graph(Edges, directed=False)

labels=[]
group=[]
for node in json_prep['nodes']:
    labels.append(node['name'])
    group.append(node['group'])
    

layt=G.layout('kk', dim=3)

Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in Edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]
    


import plotly.plotly as py
from plotly.graph_objs import *

trace1=Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=Line(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )
trace2=Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=Marker(symbol='dot',
                             size=6,
                             color=group,
                             colorscale='Viridis',
                             line=Line(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = Layout(
         title="Network of top 3 ranking pages for keyword 'SEO' (3D visualization)",
         width=1000,
         height=1000,
         showlegend=False,
         scene=Scene(
         xaxis=XAxis(axis),
         yaxis=YAxis(axis),
         zaxis=ZAxis(axis),
        ),
     margin=Margin(
        t=100
    ),
    hovermode='closest',
    annotations=Annotations([
           Annotation(
           showarrow=False,
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=Font(
            size=14
            )
            )
        ]),    )

data=Data([trace1, trace2])
fig=Figure(data=data, layout=layout)

iplot(fig, filename='Top 3 pages for keyword SEO')